In [1]:
#imports
%matplotlib inline

import numpy as np
import scipy.stats as stats
import os
import matplotlib.pyplot as plt

### Feature extraction
train will have the training samples and test will have the test samples

In [2]:
#speaker dependent partitions
#sdeptrain dimensions sdeptrain[speaker][word][utterance idx] with idxs given alphabetically
sdeptrain = [[[[] for i in range(4)] for j in range(5)] for k in range(4)]
#sdeptest dimensions sdeptrain[speaker][word] with idxs given alphabetically
sdeptest = [[[] for i in range(5)] for j in range(4)]
p = './feature'
speakerList = ['dg', 'ls', 'mh', 'yx']
wordList = ['asr','cnn','dnn','hmm','tts']
snum = 0
for s in speakerList:
    speakerpath = p+'/'+s
    wnum = 0
    for w in wordList:
        wordpath = speakerpath +'/'+s+'_'+w
        #set up train
        for utt in range(1,5):
            fpath = wordpath + str(utt)+'.fea'
            f = open(fpath,'r')
            data = f.readlines()
            data = [i.split(',') for i in data]
            for sample in range(len(data)):
                data[sample] = [float(i) for i in data[sample]]
            for d in data:
                sdeptrain[snum][wnum][utt-1].append(d)
        #set up test
        fpath2 = wordpath + str(5)+'.fea'
        f2 = open(fpath2,'r')
        data = f2.readlines()
        data = [i.split(',') for i in data]
        for d in data:
            sdeptest[snum][wnum].append(d)
        wnum += 1
    snum += 1
sdeptest = np.array(sdeptest)
sdeptrain = np.array(sdeptrain)


bigasr = []
bigcnn = []
bigdnn = []
bighmm = []
bigtts = []
for i in range(4):
    for j in range(4):
        bigasr.extend(sdeptrain[i][0][j])
        bigcnn.extend(sdeptrain[i][1][j])
        bigdnn.extend(sdeptrain[i][2][j])
        bighmm.extend(sdeptrain[i][3][j])
        bigtts.extend(sdeptrain[i][4][j])
sdepasr_cov = np.diag(np.diag(np.cov(np.array(bigasr).T)))
sdepcnn_cov = np.diag(np.diag(np.cov(np.array(bigcnn).T)))
sdepdnn_cov = np.diag(np.diag(np.cov(np.array(bigdnn).T)))
sdephmm_cov = np.diag(np.diag(np.cov(np.array(bighmm).T)))
sdeptts_cov = np.diag(np.diag(np.cov(np.array(bigtts).T)))
sdepasr_mn = np.mean(bigasr,axis=0)
sdepcnn_mn = np.mean(bigcnn,axis=0)
sdepdnn_mn = np.mean(bigdnn,axis=0)
sdephmm_mn = np.mean(bighmm,axis=0)
sdeptts_mn = np.mean(bigtts,axis=0)
print(sdepasr_mn)

[-1.41283743e+00 -1.55428945e+01  2.92278843e+00  1.49472286e-01
  6.35381972e-01  2.16369842e-01  1.42493539e-01  5.27392847e-02
  2.55955474e-01  5.94233038e-02  1.82504452e-01  4.46193938e-03
  4.64175756e-02 -1.72107996e-02]


In [80]:
#speaker independent partitions
#sindeptrain dimensions sdeptrain[speaker][word][utterance idx] with idxs given alphabetically
sindeptrain = [[[[] for i in range(5)] for j in range(5)] for k in range(3)]
#sindeptest dimensions sdeptest[word][utterance idx] with idxs given alphabetically
sindeptest = [[[] for i in range(5)] for j in range(5)]
p = './feature'
speakerList2 = ['dg', 'ls', 'mh']#'yx'
wordList = ['asr','cnn','dnn','hmm','tts']
snum = 0
for s in speakerList2:
    speakerpathi = p+'/'+s
    wnum = 0
    for w in wordList:
        wordpathi = speakerpathi +'/'+s+'_'+w
        #set up train
        for utt in range(1,6):
            fpathi = wordpathi + str(utt)+'.fea'
            fi = open(fpathi,'r')
            datai = fi.readlines()
            datai = [i.split(',') for i in datai]
            for sample in range(len(datai)):
                datai[sample] = [float(i) for i in datai[sample]]
            for d in datai:
                sindeptrain[snum][wnum][utt-1].append(d)
        wnum += 1
    snum += 1
    
        #set up test
s2 = 'yx'
speakertestpath = p+'/'+s2
wnum = 0
for w in wordList:
    wordtestpath = speakertestpath +'/'+s2+'_'+w
    for utt in range(1,6):
        ftestpath = wordtestpath + str(utt)+'.fea'
        fitest = open(ftestpath,'r')
        dataitest = fitest.readlines()
        dataitest = [i.split(',') for i in dataitest]
        for sample in range(len(dataitest)):
            dataitest[sample] = [float(i) for i in dataitest[sample]]
        for d in dataitest:
            sindeptest[wnum][utt-1].append(d)
    wnum += 1

sindeptrain = np.array(sindeptrain)
sindeptest = np.array(sindeptest)

bigasri = []
bigcnni = []
bigdnni = []
bighmmi = []
bigttsi = []
for i in range(3):
    for j in range(5):
        bigasri.extend(sindeptrain[i][0][j])
        bigcnni.extend(sindeptrain[i][1][j])
        bigdnni.extend(sindeptrain[i][2][j])
        bighmmi.extend(sindeptrain[i][3][j])
        bigttsi.extend(sindeptrain[i][4][j])
sdepasri_cov = np.diag(np.diag(np.cov(np.array(bigasri).T)))
sdepcnni_cov = np.diag(np.diag(np.cov(np.array(bigcnni).T)))
sdepdnni_cov = np.diag(np.diag(np.cov(np.array(bigdnni).T)))
sdephmmi_cov = np.diag(np.diag(np.cov(np.array(bighmmi).T)))
sdepttsi_cov = np.diag(np.diag(np.cov(np.array(bigttsi).T)))
sdepasri_mn = np.mean(bigasri,axis=0)
sdepcnni_mn = np.mean(bigcnni,axis=0)
sdepdnni_mn = np.mean(bigdnni,axis=0)
sdephmmi_mn = np.mean(bighmmi,axis=0)
sdepttsi_mn = np.mean(bigttsi,axis=0)

In [77]:
##### HELPER FUNCTIONSSSSS
#implemented from https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm

#Forward Pass

def calculateBMatrix(X, mu, sigma,N):
    #X is the data in a file
    #mu is the average across all states 
    #sigma is the covariance matrix across all states 
    #N is the number of states
    #BMatrix = [T,N]
    #print(sigma[0])
    T = len(X)
    bMatrix = np.zeros((N,T))
    for frame in range(T):
        temp_frame = []
        for state in range(N):
#             print("MUSTATE",mu[state])
#             print("COVSTATE",sigma[state])
#             print("FRAME",X[frame])
#             print(stats.multivariate_normal(mean=(mu[state]),cov=sigma[state]).pdf(X[frame]))
            bMatrix[state,frame] = stats.multivariate_normal(mean=(mu[state]),cov=sigma[state]).pdf(X[frame])
    return bMatrix

def calculateAlphaMatrix(transMatrix, BMatrix,priors):
    #transMatrix is A 
    #BMatrix is B
    #priors is pi
    #populate our alpha matrix
    T = len(BMatrix[0])
    N = len(BMatrix)
    Amat = np.array(transMatrix)
    alpha = np.zeros((N,T))
    #alphaMatrix = [T,N]
    for i in range(N):
        alpha[i,0]=priors[i]*BMatrix[i,0]
    for t in range(1,T):
        for i in range(0,N):
            alpha[i,t]=BMatrix[i,t]*np.inner(alpha[:,t-1],Amat[:,i])       
    return alpha

def calculateAlphaBetaTildeG(transMatrix,BMatrix,priors):
    T = len(BMatrix[0])
    N = len(BMatrix)
    baralpha = np.zeros((N,T))
    tildealpha = np.zeros((N,T))
    tildebeta = np.zeros((N,T))
    log_g = np.zeros((T))
    for i in range(0,N):
#         print("PI",priors[i])
#         print("B",BMatrix[i,0])
        baralpha[i,0]=priors[i]*BMatrix[i,0]
#     print(baralpha[:,0])
    log_g[0] = np.log(np.sum(baralpha[:,0]))
    tildealpha[:,0]=baralpha[:,0]/np.exp(log_g[0])

    for t in range(1,T):
        for i in range(0,N):
            baralpha[i,t]=BMatrix[i,t]*np.inner(tildealpha[:,t-1],transMatrix[:,i])
        log_g[t] = np.log(np.sum(baralpha[:,t]))
        tildealpha[:,t]=baralpha[:,t]/np.exp(log_g[t])
    for i in range(0,N):
        tildebeta[i,T-1] = 1/np.exp(log_g[T-1])

    for t in range(T-2,-1,-1):
        for i in range(0,N):
            tildebeta[i,t]=np.inner(transMatrix[i,0:N],tildebeta[:,t+1]*BMatrix[:,t+1])/np.exp(log_g[t+1])
    
    return tildealpha,tildebeta,log_g
#Backward Pass
    
def calculateBetaMatrix(transMatrix, bMatrix):
    #populate the Beta matrix
    T = len(bMatrix[0])
    N = len(bMatrix)
    beta = np.zeros((N,T))
    for i in range(N):
        beta[i,T-1]=1
    for t in range(T-2,-1,-1):
        for i in range(0,N):
            beta[i,t]=np.inner(transMatrix[i,0:N],beta[:,t+1]*bMatrix[:,t+1])
    return beta
#Update step helper functions

def calculateGammaMatrix(alphaMatrix, betaMatrix):
    T = len(betaMatrix[0])
    N = len(betaMatrix)
    gamma = np.zeros((N,T))
    for t in range(T):
       # print("ALPHA",alphaMatrix[:,t])
       # print("BETA",betaMatrix[:,t])
        gamma[:,t]=alphaMatrix[:,t]*betaMatrix[:,t]
        gamma[:,t]=gamma[:,t]/np.sum(gamma[:,t])
#     sumList = np.zeros((T))
#     for f in range(T):
#         for n in range(N):
#             sumList[f] += alphaMatrix[f][n]*betaMatrix[f][n]
#     #sumList = [sum([alphaMatrix[f][n] * betaMatrix[f][n] for n in range(N)]) for f in range(T)]
#     gammaMatrix = []
#     for frame in range(T):
#         temp_frame = []
#         for state in range(N):
#             temp_frame.append(alphaMatrix[frame][state] / sumList[frame])
#         gammaMatrix.append(temp_frame)
    return gamma

def calculateXiMatrix( transMatrix, alphaMatrix, betaMatrix, bMatrix):
    #caluculate xi 
    #probability of going from state n to state j at times t and t+1 respectively
    T = len(bMatrix[0])
    N = len(bMatrix)
    xi = np.zeros((2*N,T))
    for t in range(0,T):
        for i in range(0,N):
            for j in range(i,i+2):
                if j>= N:
                    xi[i+j,t] = 0
                else:
                    xi[i+j,t]=alphaMatrix[i,t]*transMatrix[i,j]
                if (t<T-1):
                    if j==N:
                        xi[i+j,t]=0
                    else:
                        xi[i+j,t] = xi[i+j,t]*bMatrix[j,t+1]*betaMatrix[j,t+1]
        xi[:,t]=xi[:,t]/np.sum(xi[:,t])
    
#     for frame in range(T):
#         for state in range(N):
#             for state_ in range(state, state + 2):
#                 xiMatrix[frame, state + state_] = alphaMatrix[frame, state] * transMatrix[state][state_]
#                 if frame < (T-1):
#                     if state_ == N:
#                         xiMatrix[frame, state + state_] = 0
#                     else:
#                         xiMatrix[frame, state + state_] = xiMatrix[frame, state + state_] * bMatrix[frame + 1, state_] * betaMatrix[frame+1, state_]
#         xi[:,t]=xi[:,t]/np.sum(xi[:,t])
#         xiMatrix[frame,:] = xiMatrix[frame,:]/np.sum(xiMatrix[frame, :]) 
    
    return xi 



In [75]:
#speaker independent hmm
initialProbabilities = [0.2, 0.2, 0.2, 0.2, 0.2]

initialTransition = [[0.8,0.2,  0,  0,   0],
                     [  0,0.8,0.2,  0,   0],
                     [  0,  0,0.8,0.2,   0],
                     [  0,  0,  0,0.8, 0.2],
                     [  0,  0,  0,  0,   1]]

Comments

There are N = 5 possible states and the number of observations is equal to the number of rows or time steps for each utterance. At each time step or observation, we are in one of the states, so a number 1-5 i think. So T = how many ever rows there are present. Still not sure exactly how we should chunk the data. Not sure if we should separate it by utterance or if we should group all the utterances of the same word together. Update parameters using baum welch forward/backward propogation equations. According to Piazza, update alpha and beta after each individual utterance and update the transition matrix after all the files for that word? Not how to implement this though. 

We train one hmm per word, so 5 in total. To predict a test sample, we run the utterance through each of the 5 models and assign it the label of the hmm that gives the highest probability.

In [6]:
#SDEP general
def SINDEPTRAIN(initialTransition,initialProbabilities,initialmeans,initialcov,wordidx,numiterations,numcoeff):
    A = np.array(initialTransition)
    mn = initialmeans
    cov = initialcov
    logprobs = np.zeros((15,numiterations))
    for iteration in range(numiterations):
        # calc params
        alphas = []
        betas = []
        gammas = []
        xis = []
        for s in range(3):#for each speaker
            for u in range(5): # for each utterance
                B = calculateBMatrix(sindeptrain[s][wordidx][u],mn,cov,5)
                Alpha,Beta,logg = calculateAlphaBetaTildeG(A,B,initialProbabilities)
                Gamma = calculateGammaMatrix(Alpha,Beta)
                xi = calculateXiMatrix(A,Alpha,Beta,B)
                alphas.append(Alpha)
                betas.append(Beta)
                gammas.append(Gamma)
                xis.append(xi)
                logprobs[:,iteration] = np.sum(logg)

        #upd8 
        num_files = len(alphas)
        N = len(gammas[0])
        A_new = np.zeros((N,N))
        mu_new = np.zeros((N,numcoeff))
        sigma_new = np.zeros((N,numcoeff, numcoeff))
        # find aij
        denomsum = np.zeros((N))
        for i in range(0,N):

            for f in range(num_files):
                for t in range(len(gammas[f][i])):
                    denomsum[i] += gammas[f][i][t]

            for j in range(i,i+2):
                if j<N:
                    numsum = 0
                    for f in range(num_files):
                        for t in range(len(gammas[f][i])):
                            numsum += xis[f][i+j][t]
                    A_new[i][j] = numsum/denomsum[i]
        #find mui
        for i in range(0,N):
            numsum = 0
            for f in range(num_files):
                for t in range(len(sindeptrain[f//5][wordidx][f%5])):
                    numsum += (np.array(gammas[f][i][t]) * np.array(sindeptrain[f//5][wordidx][f%5][t]))
            mu_new[i] = numsum/denomsum[i]

        #find covdi
        for i in range(0,N):
            numsum = 0
            for f in range(num_files):
                for t in range(len(sindeptrain[f//5][wordidx][f%5])):
                    numsum += (np.array(gammas[f][i][t]) * np.array(np.outer((sindeptrain[f//5][wordidx][f%5][t]-mn[i]),(sindeptrain[f//5][wordidx][f%5][t]-mn[i]))))
            sigma_new[i] = numsum/denomsum[i]

        A = A_new
        mn = mu_new
        cov = sigma_new
    print(A_new)
    print("indep hmm completed for word",wordidx)
    return A,mn,cov


In [68]:
#SDEP general
def SDEPTRAIN(initialTransition,initialProbabilities,initialmeans,initialcov,wordidx,numiterations,numcoeff):
    A = np.array(initialTransition)
    mn = initialmeans
    cov = initialcov
    logprobs = np.zeros((16,numiterations))
    for iteration in range(numiterations):
        # calc params
        alphas = []
        betas = []
        gammas = []
        xis = []
        for s in range(4):#for each speaker
            for u in range(4): # for each utterance
                B = calculateBMatrix(sdeptrain[s][wordidx][u],mn,cov,5)
                Alpha,Beta,logg = calculateAlphaBetaTildeG(A,B,initialProbabilities)
                Gamma = calculateGammaMatrix(Alpha,Beta)
                xi = calculateXiMatrix(A,Alpha,Beta,B)
                alphas.append(Alpha)
                betas.append(Beta)
                gammas.append(Gamma)
                xis.append(xi)
                logprobs[:,iteration] = np.sum(logg)

        #upd8 
        num_files = len(alphas)
        N = len(gammas[0])
        A_new = np.zeros((N,N))
        mu_new = np.zeros((N,numcoeff))
        sigma_new = np.zeros((N,numcoeff, numcoeff))
        # find aij
        denomsum = np.zeros((N))
        for i in range(0,N):

            for f in range(num_files):
                for t in range(len(gammas[f][i])):
                    denomsum[i] += gammas[f][i][t]

            for j in range(i,i+2):
                if j<N:
                    numsum = 0
                    for f in range(num_files):
                        for t in range(len(gammas[f][i])):
                            numsum += xis[f][i+j][t]
                    A_new[i][j] = numsum/denomsum[i]
        #find mui
        for i in range(0,N):

            numsum = 0
            for f in range(num_files):
                for t in range(len(gammas[f][i])):
                    numsum += (np.array(gammas[f][i][t]) * np.array(sdeptrain[f//4][wordidx][f%4][t]))
            mu_new[i] = numsum/denomsum[i]

        #find covdi
        for i in range(0,N):

            numsum = 0
            for f in range(num_files):
                for t in range(len(gammas[f][i])):
                    numsum += (np.array(gammas[f][i][t]) * np.array(np.outer((sdeptrain[f//4][wordidx][f%4][t]-mn[i]),(sdeptrain[f//4][wordidx][f%4][t]-mn[i]))))
            sigma_new[i] = numsum/denomsum[i]

        A = A_new
        mn = mu_new
        cov = sigma_new
    print(A_new)
    print("dep hmm completed for word",wordidx)
    return A,mn,cov

def testlogProb(testfile,Atest,meanstest,covtest,initProbs):
    At = np.array(Atest)
    Btest = calculateBMatrix(testfile,meanstest,covtest,5)
    logg = np.array(calculateAlphaBetaTildeG(At,Btest,initProbs)[2])
    return np.sum(logg)

In [8]:
#TRAINING FOR SPEAKER DEPENDENT
asrmean = [sdepasr_mn,sdepasr_mn,sdepasr_mn,sdepasr_mn,sdepasr_mn]
asrcova = [sdepasr_cov,sdepasr_cov,sdepasr_cov,sdepasr_cov,sdepasr_cov]
cnnmean = [sdepcnn_mn,sdepcnn_mn,sdepcnn_mn,sdepcnn_mn,sdepcnn_mn]
cnncova = [sdepcnn_cov,sdepcnn_cov,sdepcnn_cov,sdepcnn_cov,sdepcnn_cov]
dnnmean = [sdepdnn_mn,sdepdnn_mn,sdepdnn_mn,sdepdnn_mn,sdepdnn_mn]
dnncova = [sdepdnn_cov,sdepdnn_cov,sdepdnn_cov,sdepdnn_cov,sdepdnn_cov]
hmmmean = [sdephmm_mn,sdephmm_mn,sdephmm_mn,sdephmm_mn,sdephmm_mn]
hmmcova = [sdephmm_cov,sdephmm_cov,sdephmm_cov,sdephmm_cov,sdephmm_cov]
ttsmean = [sdeptts_mn,sdeptts_mn,sdeptts_mn,sdeptts_mn,sdeptts_mn]
ttscova = [sdeptts_cov,sdeptts_cov,sdeptts_cov,sdeptts_cov,sdeptts_cov]
A_asr,mn_asr,cov_asr = SDEPTRAIN(initialTransition,initialProbabilities,asrmean,asrcova,0,25,14)
A_cnn,mn_cnn,cov_cnn = SDEPTRAIN(initialTransition,initialProbabilities,cnnmean,cnncova,1,25,14)
A_dnn,mn_dnn,cov_dnn = SDEPTRAIN(initialTransition,initialProbabilities,dnnmean,dnncova,2,25,14)
A_hmm,mn_hmm,cov_hmm = SDEPTRAIN(initialTransition,initialProbabilities,hmmmean,hmmcova,3,25,14)
A_tts,mn_tts,cov_tts = SDEPTRAIN(initialTransition,initialProbabilities,ttsmean,ttscova,4,25,14)

[[0.88004837 0.11995163 0.         0.         0.        ]
 [0.         0.91842459 0.08157541 0.         0.        ]
 [0.         0.         0.89646572 0.10353428 0.        ]
 [0.         0.         0.         0.92693646 0.07306354]
 [0.         0.         0.         0.         1.        ]]
dep hmm completed for word 0
[[0.93941349 0.06058651 0.         0.         0.        ]
 [0.         0.93207714 0.06792286 0.         0.        ]
 [0.         0.         0.85274211 0.14725789 0.        ]
 [0.         0.         0.         0.94678115 0.05321885]
 [0.         0.         0.         0.         1.        ]]
dep hmm completed for word 1
[[0.85035388 0.14964612 0.         0.         0.        ]
 [0.         0.94331581 0.05668419 0.         0.        ]
 [0.         0.         0.89479096 0.10520904 0.        ]
 [0.         0.         0.         0.96392057 0.03607943]
 [0.         0.         0.         0.         1.        ]]
dep hmm completed for word 2
[[0.82018617 0.17981383 0.         0.   

In [92]:
#SPEAKER DEPENDENT TEST
#sdeptest[speaker][word]
classifications = np.zeros((5,5))
#classifications[truth][prediction] where idx{0,1,2,3,4} => {'asr','cnn','dnn','hmm','tts'}
for i in range(4):
    for j in range(5):
        file = sdeptest[i][j]
        preds = [testlogProb(file,A_asr,mn_asr,cov_asr,initialProbabilities),
                 testlogProb(file,A_cnn,mn_cnn,cov_cnn,initialProbabilities),
                 testlogProb(file,A_dnn,mn_dnn,cov_dnn,initialProbabilities),
                 testlogProb(file,A_hmm,mn_hmm,cov_hmm,initialProbabilities),
                 testlogProb(file,A_tts,mn_tts,cov_tts,initialProbabilities)]
        classifications[j][np.argmax(preds)] += 1

print(classifications/4)
print(np.mean(np.diag(classifications/4)))
    

/home/kalinuxmsi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: overflow encountered in double_scalars


[[1.   0.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   0.25 0.75 0.   0.  ]
 [0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   1.  ]]
0.95


In [93]:
#SPEAKER INDEPENDENT TRAIN
asrmeani = [sdepasri_mn,sdepasri_mn,sdepasri_mn,sdepasri_mn,sdepasri_mn]
asrcovai = [sdepasri_cov,sdepasri_cov,sdepasri_cov,sdepasri_cov,sdepasri_cov]
cnnmeani = [sdepcnni_mn,sdepcnni_mn,sdepcnni_mn,sdepcnni_mn,sdepcnni_mn]
cnncovai = [sdepcnni_cov,sdepcnni_cov,sdepcnni_cov,sdepcnni_cov,sdepcnni_cov]
dnnmeani = [sdepdnni_mn,sdepdnni_mn,sdepdnni_mn,sdepdnni_mn,sdepdnni_mn]
dnncovai = [sdepdnni_cov,sdepdnni_cov,sdepdnni_cov,sdepdnni_cov,sdepdnni_cov]
hmmmeani = [sdephmmi_mn,sdephmmi_mn,sdephmmi_mn,sdephmmi_mn,sdephmmi_mn]
hmmcovai = [sdephmmi_cov,sdephmmi_cov,sdephmmi_cov,sdephmmi_cov,sdephmmi_cov]
ttsmeani = [sdepttsi_mn,sdepttsi_mn,sdepttsi_mn,sdepttsi_mn,sdepttsi_mn]
ttscovai = [sdepttsi_cov,sdepttsi_cov,sdepttsi_cov,sdepttsi_cov,sdepttsi_cov]
A_asri,mn_asri,cov_asri = SINDEPTRAIN(initialTransition,initialProbabilities,asrmeani,asrcovai,0,25,14)
A_cnni,mn_cnni,cov_cnni = SINDEPTRAIN(initialTransition,initialProbabilities,cnnmeani,cnncovai,1,25,14)
A_dnni,mn_dnni,cov_dnni = SINDEPTRAIN(initialTransition,initialProbabilities,dnnmeani,dnncovai,2,25,14)
A_hmmi,mn_hmmi,cov_hmmi = SINDEPTRAIN(initialTransition,initialProbabilities,hmmmeani,hmmcovai,3,25,14)
A_ttsi,mn_ttsi,cov_ttsi = SINDEPTRAIN(initialTransition,initialProbabilities,ttsmeani,ttscovai,4,25,14)

[[0.87918047 0.12081953 0.         0.         0.        ]
 [0.         0.90683834 0.09316166 0.         0.        ]
 [0.         0.         0.59904418 0.40095582 0.        ]
 [0.         0.         0.         0.94039935 0.05960065]
 [0.         0.         0.         0.         1.        ]]
indep hmm completed for word 0
[[0.94716842 0.05283158 0.         0.         0.        ]
 [0.         0.4392236  0.5607764  0.         0.        ]
 [0.         0.         0.911113   0.088887   0.        ]
 [0.         0.         0.         0.9346353  0.0653647 ]
 [0.         0.         0.         0.         1.        ]]
indep hmm completed for word 1
[[0.84408361 0.15591639 0.         0.         0.        ]
 [0.         0.9095073  0.0904927  0.         0.        ]
 [0.         0.         0.88727656 0.11272344 0.        ]
 [0.         0.         0.         0.97147538 0.02852462]
 [0.         0.         0.         0.         1.        ]]
indep hmm completed for word 2
[[0.80265743 0.19734257 0.        

In [94]:
#SPEAKER INDEPENDENT TEST
#test[word][utt]
classificationsi = np.zeros((5,5))
#classifications[truth][prediction] where idx{0,1,2,3,4} => {'asr','cnn','dnn','hmm','tts'}
for i in range(5):
    for j in range(5):
        filei = sindeptest[i][j]
        predsi = [testlogProb(filei,A_asri,mn_asri,cov_asri,initialProbabilities),
                 testlogProb(filei,A_cnni,mn_cnni,cov_cnni,initialProbabilities),
                 testlogProb(filei,A_dnni,mn_dnni,cov_dnni,initialProbabilities),
                 testlogProb(filei,A_hmmi,mn_hmmi,cov_hmmi,initialProbabilities),
                 testlogProb(filei,A_ttsi,mn_ttsi,cov_ttsi,initialProbabilities)]
        classificationsi[i][np.argmax(predsi)] += 1

print(classificationsi/5)
print(np.mean(np.diag(classificationsi/5)))
  

/home/kalinuxmsi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: overflow encountered in double_scalars


[[1.  0.  0.  0.  0. ]
 [0.6 0.2 0.2 0.  0. ]
 [0.2 0.  0.8 0.  0. ]
 [0.  1.  0.  0.  0. ]
 [0.2 0.  0.  0.  0.8]]
0.5599999999999999


## Testing on Recorded Voice

In [96]:
#access voicetest[word][utt]
voicetest = [[[] for i in range(5)] for j in range(5)]
p = './feature/self_recf'
wordList = ['asr','cnn','dnn','hmm','tts']

speakertestpath = p
wnum = 0
for w in wordList:
    wordtestpath = speakertestpath +'/'+w
    for utt in range(1,6):
        ftestpath = wordtestpath + str(utt)+'.fea'
        fitest = open(ftestpath,'r')
        dataitest = fitest.readlines()
        dataitest = [i.split(',') for i in dataitest]
        for sample in range(len(dataitest)):
            dataitest[sample] = [float(i) for i in dataitest[sample]]
        for d in dataitest:
            voicetest[wnum][utt-1].append(d)
    wnum += 1

voicetest = np.array(voicetest)
#SPEAKER INDEPENDENT TEST
#test[word][utt]
classificationsvoice = np.zeros((5,5))
#classifications[truth][prediction] where idx{0,1,2,3,4} => {'asr','cnn','dnn','hmm','tts'}
for i in range(5):
    for j in range(5):
        filei = voicetest[i][j]
        predsi = [testlogProb(filei,A_asri,mn_asri,cov_asri,initialProbabilities),
                 testlogProb(filei,A_cnni,mn_cnni,cov_cnni,initialProbabilities),
                 testlogProb(filei,A_dnni,mn_dnni,cov_dnni,initialProbabilities),
                 testlogProb(filei,A_hmmi,mn_hmmi,cov_hmmi,initialProbabilities),
                 testlogProb(filei,A_ttsi,mn_ttsi,cov_ttsi,initialProbabilities)]
        classificationsvoice[i][np.argmax(predsi)] += 1
        print(predsi)

print(classificationsvoice/5)


/home/kalinuxmsi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log
/home/kalinuxmsi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in true_divide


[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


### Feature extraction for EC
For the extra credit we have opted to use the Mel Frequency Spectral Coefficients. Since the MFCC comes from taking the IDFT of the of the MFSC, we obtained the MFSC by taking the fft of the MFCC features given to us.

In [60]:
#speaker dependent partitions
#sdeptrain dimensions sdeptrain[speaker][word][utterance idx] with idxs given alphabetically
ec_sdeptrain = [[[[] for i in range(4)] for j in range(5)] for k in range(4)]
#ec_sdeptest dimensions sdeptrain[speaker][word] with idxs given alphabetically
ec_sdeptest = [[[] for i in range(5)] for j in range(4)]
p = './feature'
speakerListec = ['dg', 'ls', 'mh', 'yx']
wordListec = ['asr','cnn','dnn','hmm','tts']
snum = 0
for s in speakerListec:
    speakerpathec = p+'/'+s
    wnum = 0
    for w in wordListec:
        wordpathec = speakerpathec +'/'+s+'_'+w
        #set up train
        for utt in range(1,5):
            fpathec = wordpathec + str(utt)+'.fea'
            fec = open(fpathec,'r')
            data = fec.readlines()
            data = [i.split(',') for i in data]
            for sample in range(len(data)):
                data[sample] = [float(i) for i in data[sample]]
            for d in data:
                ec_sdeptrain[snum][wnum][utt-1].append(np.real(np.fft.fft(d)))
        #set up test
        fpath2ec = wordpathec + str(5)+'.fea'
        f2ec = open(fpath2ec,'r')
        dataec = f2ec.readlines()
        dataec = [i.split(',') for i in dataec]
        for d in dataec:
            ec_sdeptest[snum][wnum].append(np.real(np.fft.fft(np.array(d).astype(np.float))))
        wnum += 1
    snum += 1
ec_sdeptrain = np.array(ec_sdeptrain)


ec_bigasr = []
ec_bigcnn = []
ec_bigdnn = []
ec_bighmm = []
ec_bigtts = []
for i in range(4):
    for j in range(4):
        ec_bigasr.extend(ec_sdeptrain[i][0][j])
        ec_bigcnn.extend(ec_sdeptrain[i][1][j])
        ec_bigdnn.extend(ec_sdeptrain[i][2][j])
        ec_bighmm.extend(ec_sdeptrain[i][3][j])
        ec_bigtts.extend(ec_sdeptrain[i][4][j])
ec_sdepasr_cov = (np.cov(np.array(ec_bigasr).T)+0.3*np.identity(14))
ec_sdepcnn_cov = (np.cov(np.array(ec_bigcnn).T)+0.3*np.identity(14))
ec_sdepdnn_cov = (np.cov(np.array(ec_bigdnn).T)+0.3*np.identity(14))
ec_sdephmm_cov = (np.cov(np.array(ec_bighmm).T)+0.3*np.identity(14))
ec_sdeptts_cov = (np.cov(np.array(ec_bigtts).T)+0.3*np.identity(14))
ec_sdepasr_mn = np.mean(ec_bigasr,axis=0)
ec_sdepcnn_mn = np.mean(ec_bigcnn,axis=0)
ec_sdepdnn_mn = np.mean(ec_bigdnn,axis=0)
ec_sdephmm_mn = np.mean(ec_bighmm,axis=0)
ec_sdeptts_mn = np.mean(ec_bigtts,axis=0)
print(ec_sdepasr_cov)

[[ 5.23225914e+01  5.29382403e+01  3.75032229e+01  2.89647140e+01
   1.62596657e+01  1.20291698e+00 -3.31961531e+00 -7.55295367e-01
  -3.31961531e+00  1.20291698e+00  1.62596657e+01  2.89647140e+01
   3.75032229e+01  5.29382403e+01]
 [ 5.29382403e+01  5.76293407e+01  4.07127936e+01  3.11544386e+01
   1.67555639e+01  1.67746326e-02 -5.51450544e+00 -3.59749565e+00
  -5.51450544e+00  1.67746326e-02  1.67555639e+01  3.11544386e+01
   4.07127936e+01  5.73293407e+01]
 [ 3.75032229e+01  4.07127936e+01  3.15972367e+01  2.35589200e+01
   1.17420024e+01  2.51295450e-01 -4.82895211e+00 -4.62417816e+00
  -4.82895211e+00  2.51295450e-01  1.17420024e+01  2.35589200e+01
   3.12972367e+01  4.07127936e+01]
 [ 2.89647140e+01  3.11544386e+01  2.35589200e+01  2.11084869e+01
   1.10135660e+01  1.34635807e+00 -4.64283287e+00 -4.55673067e+00
  -4.64283287e+00  1.34635807e+00  1.10135660e+01  2.08084869e+01
   2.35589200e+01  3.11544386e+01]
 [ 1.62596657e+01  1.67555639e+01  1.17420024e+01  1.10135660e+01
  

In [61]:
#TRAINING FOR SPEAKER DEPENDENT FOR EXTRA CREDIT
ec_asrmean = [ec_sdepasr_mn,ec_sdepasr_mn,ec_sdepasr_mn,ec_sdepasr_mn,ec_sdepasr_mn]
ec_asrcova = [ec_sdepasr_cov,ec_sdepasr_cov,ec_sdepasr_cov,ec_sdepasr_cov,ec_sdepasr_cov]
ec_cnnmean = [ec_sdepcnn_mn,ec_sdepcnn_mn,ec_sdepcnn_mn,ec_sdepcnn_mn,ec_sdepcnn_mn]
ec_cnncova = [ec_sdepcnn_cov,ec_sdepcnn_cov,ec_sdepcnn_cov,ec_sdepcnn_cov,ec_sdepcnn_cov]
ec_dnnmean = [ec_sdepdnn_mn,ec_sdepdnn_mn,ec_sdepdnn_mn,ec_sdepdnn_mn,ec_sdepdnn_mn]
ec_dnncova = [ec_sdepdnn_cov,ec_sdepdnn_cov,ec_sdepdnn_cov,ec_sdepdnn_cov,ec_sdepdnn_cov]
ec_hmmmean = [ec_sdephmm_mn,ec_sdephmm_mn,ec_sdephmm_mn,ec_sdephmm_mn,ec_sdephmm_mn]
ec_hmmcova = [ec_sdephmm_cov,ec_sdephmm_cov,ec_sdephmm_cov,ec_sdephmm_cov,ec_sdephmm_cov]
ec_ttsmean = [ec_sdeptts_mn,ec_sdeptts_mn,ec_sdeptts_mn,ec_sdeptts_mn,ec_sdeptts_mn]
ec_ttscova = [ec_sdeptts_cov,ec_sdeptts_cov,ec_sdeptts_cov,ec_sdeptts_cov,ec_sdeptts_cov]
ec_A_asr,ec_mn_asr,ec_cov_asr = SDEPTRAIN(initialTransition,initialProbabilities,ec_asrmean,ec_asrcova,0,25,14)
ec_A_cnn,ec_mn_cnn,ec_cov_cnn = SDEPTRAIN(initialTransition,initialProbabilities,ec_cnnmean,ec_cnncova,1,25,14)
ec_A_dnn,ec_mn_dnn,ec_cov_dnn = SDEPTRAIN(initialTransition,initialProbabilities,ec_dnnmean,ec_dnncova,2,25,14)
ec_A_hmm,ec_mn_hmm,ec_cov_hmm = SDEPTRAIN(initialTransition,initialProbabilities,ec_hmmmean,ec_hmmcova,3,25,14)
ec_A_tts,ec_mn_tts,ec_cov_tts = SDEPTRAIN(initialTransition,initialProbabilities,ec_ttsmean,ec_ttscova,4,25,14)

[[0.95199724 0.04800276 0.         0.         0.        ]
 [0.         0.66372964 0.33627036 0.         0.        ]
 [0.         0.         0.89571994 0.10428006 0.        ]
 [0.         0.         0.         0.92853525 0.07146475]
 [0.         0.         0.         0.         1.        ]]
dep hmm completed for word 0
[[0.87241152 0.12758848 0.         0.         0.        ]
 [0.         0.93802442 0.06197558 0.         0.        ]
 [0.         0.         0.89885313 0.10114687 0.        ]
 [0.         0.         0.         0.94933469 0.05066531]
 [0.         0.         0.         0.         1.        ]]
dep hmm completed for word 1
[[0.84454446 0.15545554 0.         0.         0.        ]
 [0.         0.94623538 0.05376462 0.         0.        ]
 [0.         0.         0.88627527 0.11372473 0.        ]
 [0.         0.         0.         0.96392437 0.03607563]
 [0.         0.         0.         0.         1.        ]]
dep hmm completed for word 2
[[0.91130775 0.08869225 0.         0.   

In [72]:
#SPEAKER DEPENDENT TEST FOR EXTRA CREDIT
ec_classifications = np.zeros((5,5))
#classifications[truth][prediction] where idx{0,1,2,3,4} => {'asr','cnn','dnn','hmm','tts'}
for i in range(4):
    for j in range(5):
        ec_file = ec_sdeptrain[0][i][j]
        ec_preds = [testlogProb(ec_file,ec_A_asr,ec_mn_asr,ec_cov_asr,initialProbabilities),
                 testlogProb(ec_file,ec_A_cnn,ec_mn_cnn,ec_cov_cnn,initialProbabilities),
                 testlogProb(ec_file,ec_A_dnn,ec_mn_dnn,ec_cov_dnn,initialProbabilities),
                 testlogProb(ec_file,ec_A_hmm,ec_mn_hmm,ec_cov_hmm,initialProbabilities),
                 testlogProb(ec_file,ec_A_tts,ec_mn_tts,ec_cov_tts,initialProbabilities)]
        print(ec_preds)
        ec_classifications[j][np.argmax(ec_preds)] += 1

print(ec_classifications)

/home/kalinuxmsi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in log
/home/kalinuxmsi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in true_divide


[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]
[nan, nan, nan, nan, nan]


IndexError: index 4 is out of bounds for axis 0 with size 4